# Imports

In [80]:
from myosuite.simhive.myo_sim.test_sims import TestSims as loader
from scipy.signal import butter, filtfilt
from IPython.display import HTML
import matplotlib.pyplot as plt
from base64 import b64encode
import scipy.sparse as spa
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
import numpy as np
import skvideo.io
import mujoco
import osqp
import os

# Utils functions

In [81]:
def show_video(video_path, video_width = 400):
    """
    Display a video within the notebook.
    """
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")

def solve_qp(P, q, lb, ub, x0):
    """
    Solve a quadratic program.
    """
    P = spa.csc_matrix(P)
    A = spa.csc_matrix(spa.eye(q.shape[0]))
    m = osqp.OSQP()
    m.setup(P=P, q=q, A=A, l=lb, u=ub, verbose=False)
    m.warm_start(x=x0)
    res = m.solve()
    return res.x

def lowpass_filter(signal, cutoff, fs, order=5):
    """
    Low-pass filter a signal.
    """
    norm_cutoff = cutoff / (0.5 * fs)
    b, a = butter(order, norm_cutoff, btype='low', analog=False)
    return filtfilt(b, a, signal)

def plot_qxxx(qxxx, joint_names, label1, label2):
    """
    Plot generalized variables to be compared.
    qxxx[:,0,:] = time axis
    qxxx[:,1:,0] = reference sequence
    qxxx[:,1:,n] = n-th sequence
    """
    fig, axs = plt.subplots(4, 6, figsize=(12, 8))
    axs = axs.flatten()
    line_objects = []
    for j in range(1, len(joint_names)+1):
        ax = axs[j-1]
        for i in range(qxxx.shape[2]):
            line, = ax.plot(qxxx[:, 0, 0], qxxx[:, j, i])
            if j == 1: # add only one set of lines to the legend
                line_objects.append(line)
        ax.set_ylim([qxxx[:, 1:, :].min(), qxxx[:, 1:, :].max()])
        ax.set_title(joint_names[j-1])
    legend_ax = axs[len(joint_names)] # create legend in the 24th subplot area
    legend_ax.axis('off')
    labels = [label1, label2]
    legend_ax.legend(line_objects, labels, loc='center')
    plt.tight_layout()
    plt.show()

# Introduction
In this tutorial a target trajectory will be replicated by MyoHand using MuJoCo inverse dynamics, i.e., given a sequence of joint angles *qpos*, a sequence of control *ctrl* will be generated.

In [ ]:
traj = pd.read_csv('data/6_trajectory.csv').values

# Computation of the generalized force
The computation of *ctrl* is dependent on *qfrc*, which can be obtained using inverse dynamics.

In [82]:
def get_qfrc(model, data, target_qpos):
    """
    Compute the generalized force needed to reach the target position in the next mujoco step.
    """
    data_copy = deepcopy(data)
    data_copy.qpos = target_qpos
    data_copy.qvel = (target_qpos - data.qpos) / model.opt.timestep
    mujoco.mj_forward(model, data_copy)
    data_copy.qacc = 0
    mujoco.mj_inverse(model, data_copy)
    return data_copy.qfrc_inverse

The effectiveness of the computed *qfrc* can be tested by applying it directly as shown below.

In [ ]:
model0 = loader.get_sim(None, 'hand/myohand.xml')
data0 = mujoco.MjData(model0)
all_qpos = np.zeros_like(traj)
all_qfrc = np.zeros_like(traj)
for idx in (pbar := tqdm(range(traj.shape[0]))):
    target_qpos = traj[idx, 1:]
    qfrc = get_qfrc(model0, data0, target_qpos)
    data0.qfrc_applied = qfrc
    mujoco.mj_step(model0, data0)
    all_qpos[idx,:] = np.hstack((data0.time, data0.qpos))
    all_qfrc[idx,:] = np.hstack((data0.time, qfrc))
qpos_eval = np.zeros((traj.shape[0], traj.shape[1], 2))
qpos_eval[:,:,0] = traj
qpos_eval[:,:,1] = all_qpos
error = ((qpos_eval[:,1:,1] - qpos_eval[:,1:,0])**2).mean(axis=0)
print(f'Error max (rad): {error.max()}')
joint_names = [model0.joint(i).name for i in range(model0.nq)]
plot_qxxx(qpos_eval, joint_names, 'Reference qpos', 'Achieved qpos')

The difference between the reference trajectory and the achieved one is negligible. However, looking at the computed *qfrc* (e.g., shown below for mcp2_flexion over a limited time interval for the sake of visualization), its trend is highly oscillatory, and its use to compute *ctrl* would result in oscillatory and unphysiological muscle activations.

In [ ]:
x1 = 2850; x2 = 3500; qidx = 7
plt.plot(all_qfrc[x1:x2,0], all_qfrc[x1:x2,1+qidx])
plt.title(f'Obtained qfrc for {joint_names[qidx]}')
plt.show()

Therefore, *qfrc* is low-pass filtered with a cutoff frequency of 5 Hz.

In [85]:
filt_qfrc = np.zeros_like(all_qfrc)
filt_qfrc[:,0] = all_qfrc[:,0]
fs = 1/model0.opt.timestep
cutoff = 5 # Hz
filt_qfrc[:,1:] = np.apply_along_axis(lowpass_filter, 0, all_qfrc[:,1:], cutoff, fs)

The effectiveness of the filtered *qfrc* can be tested below. The obtained error is slightly higher than the previous one, but still negligible. The performance variation w.r.t. cutoff frequency changes is up to the reader.

In [ ]:
model0 = loader.get_sim(None, 'hand/myohand.xml')
data0 = mujoco.MjData(model0)
all_qpos = np.zeros_like(traj)
for idx in (pbar := tqdm(range(traj.shape[0]))):
    data0.qfrc_applied = filt_qfrc[idx, 1:]
    mujoco.mj_step(model0, data0)
    all_qpos[idx,:] = np.hstack((data0.time, data0.qpos))
qpos_eval[:,:,1] = all_qpos
error = ((qpos_eval[:,1:,1] - qpos_eval[:,1:,0])**2).mean(axis=0)
print(f'Error max (rad): {error.max()}')
plot_qxxx(qpos_eval, joint_names, 'Reference qpos', 'Achieved qpos')

# Compuation of the control
The following describes how the ***get_ctrl*** function works.
The equation to solve for *ctrl*, accordingly to muscle actuators dynamics (https://mujoco.readthedocs.io/en/latest/modeling.html#muscles), is:
$$
AM\cdot\left(gain\odot\left(act+timestep\cdot\frac{ctrl-act}{\tau}\right)+bias\right)-qfrc=0\tag{1}
$$
where
$$
\tau=\tau_D+(\tau_A-\tau_D)\cdot sigmoid\left(\frac{ctrl-act}{tausmooth}+0.5\right)
$$
To find a solution quickly, it is better to reformulate the equation and solve a quadratic program (QP), i.e.:
$$
\min_{x} \frac{1}{2}x^TPx+q^Tx~~~s.t.~~~lb\leq x\leq ub\tag{2}
$$
The major obstacle to this formulation is the sigmoid in the calculation of $\tau$. MuJoCo implements the sigmoid using the polynomial $6x^5-15x^4+10x^3$ clipped between 0 and 1. To solve the QP, the sigmoid is approximated here with $1.875x-0.4375$ and, to limit its range for a good approximation, *tausmooth* is set to 5. The target equation can then be rewritten as:
$$
AM\cdot\left(gain\odot\left(act+timestep\cdot\frac{ctrl-act}{(ctrl-act)\cdot \tau_1+\tau_2}\right)+bias\right)-qfrc=0\tag{3}
$$
where
$$
\tau_1=\frac{\tau_A-\tau_D}{tausmooth}\cdot 1.875,~~~\tau_2=(\tau_A+\tau_D)\cdot 0.5
$$
and consequently reformulated as:
$$
AM\cdot x+k=0\tag{4}
$$
where
$$
x=\left(timestep\cdot gain\odot\frac{ctrl-act}{(ctrl-act)\cdot \tau_1+\tau_2}\right),~~~k=AM\cdot(gain\odot act)+AM\cdot bias-qfrc
$$
Referring to equation $(2)$ then:
$$
P=2\cdot AM^T\cdot AM\tag{5}
$$
$$
q=2\cdot AM^T\cdot k\tag{6}
$$
$$
lb=timestep\cdot gain\odot\frac{1-act}{(1-act)\cdot \tau_1 + \tau_2}\tag{7}
$$
$$
ub=timestep\cdot gain\odot\frac{-act}{-act\cdot \tau_1 + \tau_2}\tag{8}
$$
After solving the QP for *x*, *ctrl* is then calculated as:
$$
ctrl = act + \frac{x\cdot\tau_2}{timestep\cdot gain-x\cdot\tau_1}\tag{9}
$$

In [60]:
def get_ctrl(model, data, target_qpos, qfrc):
    """
    Compute the control needed to reach the target position in the next mujoco step.
    qfrc: generalized force resulting from inverse dynamics.
    """
    act = data.act
    ctrl0 = data.ctrl
    ts = model.opt.timestep
    tA = model.actuator_dynprm[:,0] * (0.5 + 1.5 * act)
    tD = model.actuator_dynprm[:,1] / (0.5 + 1.5 * act)
    tausmooth = model.actuator_dynprm[:,2]
    # ---- gain, bias, and moment computation
    data_copy = deepcopy(data)
    data_copy.qpos = target_qpos
    data_copy.qvel = (target_qpos - data.qpos) / model.opt.timestep
    mujoco.mj_step1(model, data_copy) # gain, bias, and moment depend on qpos and qvel
    gain = np.array([])
    bias = np.array([])
    for idx_actuator in range(model.nu):
        length = data_copy.actuator_length[idx_actuator]
        lengthrange = model.actuator_lengthrange[idx_actuator]
        velocity = data_copy.actuator_velocity[idx_actuator]
        acc0 = model.actuator_acc0[idx_actuator]
        prmb = model.actuator_biasprm[idx_actuator,:9]
        prmg = model.actuator_gainprm[idx_actuator,:9]
        bias = np.append(bias, mujoco.mju_muscleBias(length, lengthrange, acc0, prmb))
        gain = np.append(gain, min(-1, mujoco.mju_muscleGain(length, velocity, lengthrange, acc0, prmg)))
    AM = data_copy.actuator_moment.T
    # ---- ctrl computation
    t1 = (tA - tD) * 1.875 / tausmooth
    t2 = (tA + tD) * 0.5
    P = 2 * AM.T @ AM
    k = AM @ (gain * act) + AM @ bias - qfrc
    q = 2 * k @ AM
    lb = gain * (1 - act) * ts / (t2 + t1 * (1 - act))
    ub = - gain * act * ts / (t2 - t1 * act)
    x0 = (gain * (ctrl0 - act) * ts) / ((ctrl0 - act) * t1 + t2)
    x = solve_qp(P, q, lb, ub, x0)
    ctrl = act + x * t2 / (gain * ts - x * t1)
    return np.clip(ctrl,0,1)

The *ctrl* sequence to be applied to MyoHand in order to replicate the reference trajectory can thus be achieved.

In [45]:
model1 = loader.get_sim(None, 'hand/myohand.xml')
tausmooth = 5
model1.actuator_dynprm[:,2] = tausmooth
data1 = mujoco.MjData(model1)
all_ctrl = np.zeros((traj.shape[0], 1+model1.nu))
for idx in (pbar := tqdm(range(traj.shape[0]))):
    target_qpos = traj[idx, 1:]
    qfrc = filt_qfrc[idx, 1:]
    ctrl = get_ctrl(model1, data1, target_qpos, qfrc)
    data1.ctrl = ctrl
    mujoco.mj_step(model1, data1)
    all_ctrl[idx,:] = np.hstack((data1.time, ctrl))

# Results
The trajectory achievable by applying the *ctrl* sequence resulting from the previous block is compared below with the reference trajectory.

In [ ]:
# ---- initializations
model_ref = loader.get_sim(None, 'hand/myohand.xml')
model_ref.actuator_dynprm[:,2] = tausmooth
data_ref = mujoco.MjData(model_ref) # data for reference trajectory
model_test = loader.get_sim(None, 'hand/myohand.xml')
model_test.actuator_dynprm[:,2] = tausmooth
data_test = mujoco.MjData(model_test) # test data for achieved trajectory
# ---- camera settings
camera = mujoco.MjvCamera()
camera.azimuth = 166.553
camera.distance = 1.178
camera.elevation = -36.793
camera.lookat = np.array([-0.93762553, -0.34088276, 0.85067529])
options_ref = mujoco.MjvOption()
options_ref.flags[:] = 0
options_ref.geomgroup[1:] = 0
options_test = mujoco.MjvOption()
options_test.flags[:] = 0
options_test.flags[4] = 1 # actuator ON
options_test.geomgroup[1:] = 0
renderer_ref = mujoco.Renderer(model_ref)
renderer_ref.scene.flags[:] = 0
renderer_test = mujoco.Renderer(model_test)
renderer_test.scene.flags[:] = 0
# ---- generation loop
frames = []
all_qpos = np.zeros_like(traj)
for idx in (pbar := tqdm(range(traj.shape[0]))):
    # -- reference trajectory
    data_ref.qpos = traj[idx, 1:]
    mujoco.mj_step(model_ref, data_ref)
    renderer_ref.update_scene(data_ref, camera=camera, scene_option=options_ref)
    frame_ref = renderer_ref.render()
    # -- achieved trajectory
    data_test.ctrl = all_ctrl[idx, 1:]
    mujoco.mj_step(model_test, data_test)
    all_qpos[idx,:] = np.hstack((data_test.time, data_test.qpos))
    renderer_test.update_scene(data_test, camera=camera, scene_option=options_test)
    frame_test = renderer_test.render()
    # -- frames merging 
    frame_merged = np.append(frame_ref, frame_test, axis=1)
    frames.append(frame_merged)
# -- frames writing
os.makedirs('videos', exist_ok = True)
output_name = 'videos/myohand_freemovement.mp4'
skvideo.io.vwrite(output_name, np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})

In [ ]:
show_video(output_name)

Left, reference trajectory. Right, achieved trajectory.